In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

spectrum_train = pd.read_excel("spectrum_train.xlsx")
spectrum_valid = pd.read_excel("spectrum_valid.xlsx")
temp_train = pd.read_excel("temp_train.xlsx")
temp_valid = pd.read_excel("temp_valid.xlsx")

In [7]:
temp_train

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,layer 11
0,350.642411,357.423984,338.000000,327.423984,320.642411,310.892016,297.633687,282.961391,267.997532,252.999904,237.999998
1,352.187496,360.362911,342.045085,332.179267,325.642411,315.647298,301.678772,285.900317,269.542617,252.999904,236.454913
2,350.642411,357.423984,338.000000,327.423984,320.642411,310.892016,297.633687,282.961391,267.997532,252.999904,237.999998
3,356.822751,369.179689,354.180340,346.445115,340.642411,329.913146,313.814027,294.717096,274.177872,252.999904,231.819658
4,340.957124,353.784211,338.000000,323.784211,310.957124,299.046473,287.454152,275.642411,263.261445,250.182832,236.453905
...,...,...,...,...,...,...,...,...,...,...,...
1291,599.180436,589.758173,579.218949,567.387443,555.107984,544.378719,534.756356,519.755705,494.756356,495.357589,498.927645
1292,595.817168,582.738555,570.357589,558.545848,546.953527,535.042876,522.215789,508.000000,492.215789,505.042876,516.953527
1293,597.362253,585.677481,574.402674,563.301131,551.953527,539.798158,526.260874,510.938926,493.760874,505.042876,515.408442
1294,595.817168,582.738555,570.357589,558.545848,546.953527,535.042876,522.215789,508.000000,492.215789,505.042876,516.953527


In [8]:
spectrum_train.columns

Float64Index([4.75e-06,  4.8e-06, 4.85e-06,  4.9e-06, 4.95e-06,    5e-06,
              5.05e-06,  5.1e-06, 5.15e-06,  5.2e-06, 5.25e-06,  5.3e-06,
              5.35e-06,  5.4e-06, 5.45e-06,  5.5e-06, 5.55e-06,  5.6e-06,
              5.65e-06,  5.7e-06, 5.75e-06,  5.8e-06, 5.85e-06,  5.9e-06,
              5.95e-06,    6e-06, 6.05e-06,  6.1e-06, 6.15e-06,  6.2e-06,
              6.25e-06,  6.3e-06, 6.35e-06,  6.4e-06, 6.45e-06,  6.5e-06,
              6.55e-06,  6.6e-06, 6.65e-06,  6.7e-06, 6.75e-06,  6.8e-06,
              6.85e-06,  6.9e-06, 6.95e-06,    7e-06, 7.05e-06,  7.1e-06,
              7.15e-06,  7.2e-06, 7.25e-06,  7.3e-06, 7.35e-06,  7.4e-06,
              7.45e-06,  7.5e-06, 7.55e-06,  7.6e-06, 7.65e-06,  7.7e-06,
              7.75e-06,  7.8e-06, 7.85e-06,  7.9e-06, 7.95e-06,    8e-06],
             dtype='float64')

In [17]:
temp_train.iloc[0].tolist()

[350.642411176571,
 357.423984338572,
 338.0,
 327.423984338572,
 320.642411176571,
 310.892015508763,
 297.633687222225,
 282.961390917275,
 267.997531803918,
 252.999904297652,
 237.999997749297]